# உற்பத்தியில் AutoGen முகவர்கள்: கண்காணிப்பு மற்றும் மதிப்பீடு

இந்த பாடத்தில், **[Autogen agents](https://github.com/microsoft/autogen)**-இன் உள்நிலை செயல்பாடுகளை (traces) **கண்காணிக்க** மற்றும் அதன் செயல்திறனை **மதிப்பீடு செய்ய** [Langfuse](https://langfuse.com) பயன்படுத்துவது எப்படி என்பதை நாம் கற்றுக்கொள்வோம்.

இந்த வழிகாட்டி **ஆன்லைன்** மற்றும் **ஆஃப்லைன்** மதிப்பீட்டு அளவுகோல்களை உள்ளடக்குகிறது, இது குழுக்களுக்கு முகவர்களை விரைவாகவும் நம்பகமாகவும் உற்பத்தியில் கொண்டு வர உதவுகிறது.

**ஏன் AI முகவர் மதிப்பீடு முக்கியம்:**
- பணிகள் தோல்வியடையும் போது அல்லது குறைவான முடிவுகளை உருவாக்கும் போது பிரச்சினைகளை சரிசெய்தல்
- நேரடி நேரத்தில் செலவுகள் மற்றும் செயல்திறனை கண்காணித்தல்
- தொடர்ச்சியான கருத்துக்களூடாக நம்பகத்தன்மை மற்றும் பாதுகாப்பை மேம்படுத்துதல்


## படி 1: சூழல் மாறிகள் அமைக்கவும்

உங்கள் Langfuse API கீகளை பெற [Langfuse Cloud](https://cloud.langfuse.com/) இல் பதிவு செய்யவும் அல்லது [Langfuse ஐ சுயமாக ஹோஸ்ட் செய்யவும்](https://langfuse.com/self-hosting).

_**குறிப்பு:** சுயமாக ஹோஸ்ட் செய்பவர்கள் [Terraform modules](https://langfuse.com/self-hosting/azure) ஐ பயன்படுத்தி Azure இல் Langfuse ஐ நிறுவலாம். மாற்றாக, [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm) ஐ பயன்படுத்தி Kubernetes இல் Langfuse ஐ நிறுவலாம்._


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

In [1]:
!pip install langfuse openlit


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


சுற்றுச்சூழல் மாறிகள் அமைக்கப்பட்ட பிறகு, நாம் இப்போது Langfuse கிளையண்டை தொடங்கலாம். `get_client()` சுற்றுச்சூழல் மாறிகளில் வழங்கப்பட்ட சான்றுகளைப் பயன்படுத்தி Langfuse கிளையண்டை தொடங்குகிறது.


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## படி 2: OpenLit கருவியை தொடங்குதல்

இப்போது, [OpenLit](https://github.com/openlit/openlit) கருவியை தொடங்குகிறோம். OpenLit தானாகவே AutoGen செயல்பாடுகளைப் பதிவு செய்து OpenTelemetry (OTel) spans-ஐ Langfuse-க்கு ஏற்றுமதி செய்கிறது.


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## படி 3: உங்கள் முகவரியை இயக்கவும்

இப்போது நாங்கள் எங்கள் கருவிகளை சோதிக்க ஒரு பல முறை முகவரியை அமைக்கிறோம்.


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### தடம் அமைப்பு

Langfuse ஒரு **தடத்தை** பதிவு செய்கிறது, இது **விரிவுகளை** கொண்டுள்ளது, உங்கள் முகவரியின் தர்க்கத்தின் ஒவ்வொரு படியையும் பிரதிநிதித்துவம் செய்கிறது. இங்கே, தடம் மொத்த முகவரி இயக்கத்தையும் மற்றும் கீழ்க்காணும் துணை-விரிவுகளையும் கொண்டுள்ளது:
- உணவுத் திட்டமிடும் முகவர்
- ஊட்டச்சத்து நிபுணர்கள்

இவற்றை ஆய்வு செய்து நேரம் எங்கு செலவிடப்படுகிறது, எத்தனை டோக்கன்கள் பயன்படுத்தப்படுகின்றன என்பதைக் கண்டறியலாம்:

![Langfuse இல் தட மரம்](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[தடத்திற்கான இணைப்பு](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## ஆன்லைன் மதிப்பீடு

ஆன்லைன் மதிப்பீடு என்பது ஒரு நேரடி, நிஜ உலக சூழலில், அதாவது உற்பத்தியில் உண்மையான பயன்பாட்டின் போது, முகவரின் செயல்திறனை மதிப்பீடு செய்வதைக் குறிக்கிறது. இது உண்மையான பயனர் தொடர்புகளில் முகவரின் செயல்திறனை கண்காணித்து, முடிவுகளை தொடர்ந்து பகுப்பாய்வு செய்வதை உள்ளடக்கியது.

### உற்பத்தியில் கண்காணிக்க வேண்டிய பொதுவான அளவுகோல்கள்

1. **செலவுகள்** — கருவிகள் டோக்கன் பயன்பாட்டை பதிவு செய்கின்றன, இதை டோக்கனுக்கு ஒரு விலை ஒதுக்கி சுமார் செலவுகளாக மாற்றலாம்.
2. **தாமதம்** — ஒவ்வொரு படியையும் அல்லது முழு செயல்பாட்டையும் முடிக்க எடுக்கும் நேரத்தை கவனிக்கவும்.
3. **பயனர் கருத்து** — பயனர்கள் நேரடி கருத்துக்களை (தொடர்பானது/தொடர்பில்லாதது) வழங்கி முகவரை மேம்படுத்த அல்லது திருத்த உதவலாம்.
4. **LLM-as-a-Judge** — உங்கள் முகவரின் வெளியீட்டை நேரடி நேரத்தில் மதிப்பீடு செய்ய தனி LLM ஐ பயன்படுத்தவும் (உதாரணமாக, நச்சுத்தன்மை அல்லது சரியானதா என்பதை சரிபார்க்க).

கீழே, இந்த அளவுகோள்களின் உதாரணங்களை காணலாம்.


#### 1. செலவுகள்

கீழே `gpt-4o-mini` அழைப்புகளுக்கான பயன்பாட்டை காட்டும் ஒரு ஸ்கிரீன்ஷாட் உள்ளது. இது அதிக செலவான படிகளை கண்டறிந்து உங்கள் ஏஜெண்டை மேம்படுத்த உதவுகிறது.

![செலவுகள்](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[டிரேஸுக்கு இணைப்பு](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. தாமதம்

ஒவ்வொரு படியையும் முடிக்க எவ்வளவு நேரம் எடுத்துக்கொண்டது என்பதை நாமும் காணலாம். கீழே உள்ள உதாரணத்தில், முழு செயல்பாடும் சுமார் 3 விநாடிகளில் முடிந்தது, இதை நீங்கள் ஒவ்வொரு படியாக பிரிக்கலாம். இது bottlenecks-ஐ அடையாளம் காணவும், உங்கள் முகவரியை மேம்படுத்தவும் உதவுகிறது.

![தாமதம்](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[டிரேஸுக்கு இணைப்பு](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. பயனர் கருத்து

உங்கள் ஏஜென்ட் ஒரு பயனர் இடைமுகத்தில் இணைக்கப்பட்டிருந்தால், நேரடி பயனர் கருத்துகளை (உதாரணமாக, ஒரு அரட்டை UI-யில் விருப்பம்/விருப்பமின்மை போன்றவை) பதிவு செய்யலாம்.


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

பயனர் கருத்துகள் பின்னர் Langfuse-ல் பதிவு செய்யப்படுகின்றன:

![பயனர் கருத்துகள் Langfuse-ல் பதிவு செய்யப்படுகின்றன](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. தானியக்க LLM-as-a-Judge மதிப்பீடு

LLM-as-a-Judge என்பது உங்கள் முகவரியின் வெளியீட்டை தானியக்கமாக மதிப்பீடு செய்யும் மற்றொரு வழியாகும். வெளியீட்டின் சரியான தன்மை, விஷமத்தன்மை, பாணி அல்லது உங்களுக்கு முக்கியமான பிற அளவுகோள்களை மதிப்பீடு செய்ய தனி LLM அழைப்பை அமைக்கலாம்.

**செயல்முறை**:
1. நீங்கள் ஒரு **மதிப்பீட்டு வார்ப்புருவை** வரையறுக்கிறீர்கள், உதாரணமாக, "உரையில் விஷமத்தன்மை உள்ளதா என்பதை சரிபார்க்கவும்."
2. நீங்கள் ஒரு மாடலை நியமிக்கிறீர்கள், இது judge-model ஆக செயல்படும்; இங்கு `gpt-4o-mini` Azure மூலம் கேட்கப்படுகிறது.
3. உங்கள் முகவரி ஒவ்வொரு முறையும் வெளியீட்டை உருவாக்கும் போது, அந்த வெளியீட்டை "நீதிபதி" LLM-க்கு வார்ப்புருவுடன் அனுப்புகிறீர்கள்.
4. நீதிபதி LLM ஒரு மதிப்பீடு அல்லது லேபலை பதிலளிக்கிறது, அதை நீங்கள் உங்கள் கண்காணிப்பு கருவியில் பதிவு செய்கிறீர்கள்.

Langfuse-இல் இருந்து எடுத்துக்காட்டு:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

இந்த எடுத்துக்காட்டின் பதில் "விஷமில்லாதது" என்று மதிப்பீடு செய்யப்பட்டுள்ளது என்பதை நீங்கள் காணலாம்.

![LLM-as-a-Judge மதிப்பீட்டு மதிப்பெண்](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### 5. கண்காணிப்பு அளவுகோள்களின் மேற்பார்வை

இந்த அளவுகோள்களை அனைத்தையும் டாஷ்போர்ட்களில் ஒன்றாக காணலாம். இது உங்கள் ஏஜென்ட் பல அமர்வுகளில் எப்படி செயல்படுகிறது என்பதை விரைவாக பார்க்க உதவுகிறது மற்றும் தர அளவுகோள்களை காலத்திற்கேற்ப கண்காணிக்க உதவுகிறது.

![கண்காணிப்பு அளவுகோள்களின் மேற்பார்வை](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## ஆஃப்லைன் மதிப்பீடு

ஆன்லைன் மதிப்பீடு நேரடி கருத்துகளை பெற முக்கியமானது, ஆனால் உங்களுக்கு **ஆஃப்லைன் மதிப்பீடு** கூட தேவை—உருமாற்றம் செய்யும் முன்பும் அல்லது வளர்ச்சியின் போது செய்யப்படும் முறையான சரிபார்ப்புகள். இது மாற்றங்களை உற்பத்தியில் கொண்டு வருவதற்கு முன் தரம் மற்றும் நம்பகத்தன்மையை பராமரிக்க உதவுகிறது.


### தரவுத்தொகுப்பு மதிப்பீடு

ஆஃப்லைன் மதிப்பீட்டில், நீங்கள் பொதுவாக:
1. ஒரு தரநிலை தரவுத்தொகுப்பை (உருவாக்கம் மற்றும் எதிர்பார்க்கப்படும் வெளியீடு ஜோடிகளுடன்) வைத்திருப்பீர்கள்
2. உங்கள் முகவரியை அந்த தரவுத்தொகுப்பில் இயக்குவீர்கள்
3. வெளியீடுகளை எதிர்பார்க்கப்படும் முடிவுகளுடன் ஒப்பிடுவீர்கள் அல்லது கூடுதல் மதிப்பீட்டு முறையைப் பயன்படுத்துவீர்கள்

கீழே, [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) உடன் இந்த அணுகுமுறையை விளக்குகிறோம், இது கேள்விகள் மற்றும் எதிர்பார்க்கப்படும் பதில்களை கொண்டுள்ளது.


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

அடுத்ததாக, Langfuse-ல் ஒரு தரவுத்தொகுப்பு அமைப்பை உருவாக்கி இயக்கங்களை கண்காணிக்கிறோம். பின்னர், தரவுத்தொகுப்பில் உள்ள ஒவ்வொரு உருப்படியையும் அமைப்பில் சேர்க்கிறோம்.


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Dataset items in Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### தரவுத்தொகுப்பில் முகவரியை இயக்குதல்

முதலில், Azure OpenAI மாதிரிகளைப் பயன்படுத்தி கேள்விகளுக்கு பதிலளிக்கும் ஒரு எளிய Autogen முகவரியை உருவாக்குகிறோம்.


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

பின்னர், நாங்கள் `my_agent()` என்ற உதவி செயல்பாட்டை வரையறுக்கிறோம்.


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

இறுதியாக, ஒவ்வொரு தரவுத்தொகுப்பு உருப்படியையும் மீண்டும் மீண்டும் செயல்படுத்தி, முகவரியை இயக்கி, தடத்தை தரவுத்தொகுப்பு உருப்படிக்கு இணைக்கிறோம். தேவையெனில், ஒரு விரைவான மதிப்பீட்டு மதிப்பையும் இணைக்கலாம்.


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

நீங்கள் இந்த செயல்முறையை மாறுபட்ட ஏஜென்ட் அமைப்புகளுடன் மீண்டும் செய்யலாம், உதாரணமாக:
- மாடல்கள் (gpt-4o-mini, gpt-4.1, போன்றவை)
- ப்ராம்ப்ட்கள்
- கருவிகள் (தேடல் vs. தேடல் இல்லாமல்)
- ஏஜென்டின் சிக்கலான தன்மை (பல ஏஜென்ட் vs. ஒற்றை ஏஜென்ட்)

பின்னர் அவற்றை Langfuse-ல் பக்கப்பக்கமாக ஒப்பிடலாம். இந்த எடுத்துக்காட்டில், நான் 25 தரவுத்தொகுப்பு கேள்விகளில் ஏஜென்டை 3 முறை இயக்கினேன். ஒவ்வொரு இயக்கத்திற்கும், நான் வேறு Azure OpenAI மாடலை பயன்படுத்தினேன். பெரிய மாடலைப் பயன்படுத்தும்போது சரியாக பதிலளிக்கப்பட்ட கேள்விகளின் அளவு அதிகரிக்கிறது என்பதை நீங்கள் காணலாம் (எதிர்பார்த்தபடி). `correct_answer` மதிப்பெண் [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) மூலம் உருவாக்கப்படுகிறது, இது தரவுத்தொகுப்பில் கொடுக்கப்பட்ட மாதிரி பதிலின் அடிப்படையில் கேள்வியின் சரியானதன்மையை மதிப்பீடு செய்ய அமைக்கப்பட்டுள்ளது.

![தரவுத்தொகுப்பு இயக்கத்தின் மேற்பார்வை](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![தரவுத்தொகுப்பு இயக்கத்தின் ஒப்பீடு](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
